In [ ]:
import pandas as pd
import requests
from tkinter import *
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
#Establish URL
url = 'https://coinmarketcap.com/'

In [ ]:
#Say Hi to the URL
browser.visit(url)

In [ ]:
tokens = []

for i in range(1, 4, 1):
    loop_url = f'https://coinmarketcap.com/?page={i}'
    browser.visit(loop_url)
    
    for j in range(10):
        browser.execute_script("window.scrollBy(0,1000);")
        time.sleep(.5)
        
    html = browser.html
    
    soup = BeautifulSoup(html, 'html.parser')
    
    table = soup.find('table', class_='cmc-table')
    trs = table.find_all('tr')
    
    print(f'********Page: {i}')

    for tr in trs:

        token = {}

        tds = tr.find_all('td')
        
        try:
            if tds[2].p is not None:
                coin_name = tds[2].p.text
                coin_code = tds[2].div.div.div.p.text
            else:
                coin_name = tds[2].find_all('span')[1].text
                coin_code = tds[2].find_all('span')[2].text

            coin_price = tds[3].text.replace(',','').strip('$')
            coin_24h = tds[4].text.strip('%')
            coin_7d = tds[5].text.strip('%')
            coin_market_cap = tds[6].text.replace(',','').strip('$')
            coin_volume_usd = tds[7].find_all('p')[0].text.replace(',','').strip('$')
            coin_volume_crypto_list = tds[7].find_all('p')[1].text.replace(',','').strip('$').split()
            coin_volume = coin_volume_crypto_list[0]
            coin_volume_token = coin_volume_crypto_list[1]
            coin_circulating_supply_list = tds[8].text.replace(',','').strip('$').split()
            coin_circulating_supply = coin_circulating_supply_list[0]
            coin_circulating_supply_token = coin_circulating_supply_list[1]

            # set up dictionary record
            token['token'] = coin_code
            token['name'] = coin_name
            token['price'] = coin_price
            token['24h'] = coin_24h
            token['7d'] = coin_7d
            token['market_cap'] = coin_market_cap
            token['volume_usd'] = coin_volume_usd
            token['volume'] = coin_volume
            token['circulating_supply'] = coin_circulating_supply

            # add to the list of dictionaries
            tokens.append(token)

        except Exception as e:
            i = trs.index(tr)
            print(f'Error with index [{i}] of the tr.')
            continue
            
browser.quit()

In [ ]:
#Create DataFrame
token = pd.DataFrame(tokens)
token

In [ ]:
#Extract Master array of NAMES ONLY for all Cryptos
coins = token['name'].values
#coins

<h1>Query<h1>

In [ ]:
#Setting Up Drop Down Function
root = Tk()
root.geometry( "200x200" )
def show():
    label.config( text = clicked.get() )
    
options = coins
clicked = StringVar()
clicked.set('Click Me!')

drop = OptionMenu( root, clicked, *options)
drop.pack()

button = Button(root, text = 'Select This Coin?', command =show).pack()

label= Label(root, text = " ")
label.pack()

root.mainloop()

content = clicked.get()
#content
crypto = content
crypto

#Don't Forget to 'x' out of the prompt!

In [ ]:
#FORMAT QUERY
lowercase = str.lower(crypto)
formed = lowercase.replace(" ","-")
#formed
exchange = f'https://www.coindesk.com/price/{formed}'
#exchange

<h1>CoinDesk | First Test<h1>

In [ ]:
#Stir the pot
# trade = requests.get(exchange)
# soup = BeautifulSoup(trade.text, 'html.parser')
# soup

In [ ]:
#You Spin My Head Right Round
# Title = ''
# Price = ''
# Change24 = ''
# MarketCap = ''
# Volume = ''
# Low24 = ''
# High24 = ''
# NetChange = ''
# Open24 = ''
# AllTimeHigh = ''
# TotalSupply= ''
# Returns24 = ''
# ReturnsYTD = ''
# Volatility30Day = ''
# TransactionCount24 = ''
# AvgTransFee24 = ''
# ValueTransferred24 = ''
# About = ''
# try:
#     Title = soup.select('div.coin-title .heading')[0].text
#     Price = soup.select('div.coin-info-block .data-definition')[0].text
#     Change24 = soup.select('div.coin-info-block .data-definition')[1].text
#     MarketCap = soup.select('div.coin-info-block .data-definition')[2].text
#     Volume = soup.select('div.coin-info-block .data-definition')[3].text
#     Low24 = soup.select('div.coin-info-block .data-definition')[4].text
#     High24 = soup.select('div.coin-info-block .data-definition')[5].text
#     Open24 = soup.select('div.coin-info-block .data-definition')[6].text
#     Open24 = soup.select('div.coin-info-block .data-definition')[7].text 
#     AllTimeHigh = soup.select('div.coin-info-block .data-definition')[8].text
#     TotalSupply= soup.select('div.coin-info-block .data-definition')[9].text
#     Returns24 = soup.select('div.coin-info-block .data-definition')[10].text
#     ReturnsYTD = soup.select('div.coin-info-block .data-definition')[11].text
#     Volatility30Day = soup.select('div.coin-info-block .data-definition')[12].text
#     TransactionCount24 = soup.select('div.coin-info-block .data-definition')[13].text
#     AvgTransFee24 = soup.select('div.coin-info-block .data-definition')[14].text
#     ValueTransferred24 = soup.select('div.coin-info-block .data-definition')[15].text
#     About = soup.select('div.coin-about-text')[0].text
# except:
#     print('Crypto Not Found')

In [ ]:
#DUMP IT ALL
# print(f'---------------{Title}--------------------')
# print(f'Price : {Price}')
# print(f'24hr % Change : {Change24}')
# print(f'Market Cap : {MarketCap}')
# print(f'24hr Volume : {Volume}')
# print(f'24hr Low : {Low24}')
# print(f'24hr High : {High24}')
# print(f'Net Change : {NetChange}')
# print('---------------Metrics--------------------')
# print(f'24hr Open : {Open24}')
# print(f'All Time High : {AllTimeHigh}')
# print(f'Total Supply : {TotalSupply}')
# print(f'24hr Returns : {Returns24}')
# print(f'Year To Date Returns : {ReturnsYTD}')
# print(f'30 Day Volatility : {Volatility30Day}')
# print(f'24hr Transaction Count : {TransactionCount24}')
# print(f'24hr Average Transaction Fee {AvgTransFee24}')
# print(f'24hr Value Transacted : {ValueTransferred24}')
# print('---------------About--------------------')
# print(About)
# print('---------------Source--------------------')
# print(exchange)

<h1>CoinMarketCap | More Reliable<h1>

In [ ]:
#COINMARKET PULL (BETTER SUCCESS)
wall_street = f'https://coinmarketcap.com/currencies/{formed}/'
print(wall_street)

In [ ]:
#Stir the pot
call = requests.get(wall_street)
chowder = BeautifulSoup(call.text, 'html.parser')
chowder

In [ ]:
#You Spin My Head Right Round
Title = ''
Price = ''
Change24 = ''
MarketCap = ''
Volume = ''
Low24 = ''
About = ''
try:
    Title = chowder.select('small.nameSymbol___1arQV')[0].text
    Price = chowder.select('div.priceValue___11gHJ')[0].text
    Change24 = chowder.select('.sc-1v2ivon-0')[0].text
    MarketCap = chowder.select('div.statsValue___2iaoZ')[0].text
    Volume = chowder.select('div.statsValue___2iaoZ')[2].text
    Low24 = chowder.select('.highLowValue___GfyK7')[0].text
    High24 = chowder.select('.highLowValue___GfyK7')[1].text
    About = chowder.select('div.about___1OuKY')[0].text
except:
    print('Crypto Not Found')

In [ ]:
#DUMP IT ALL
print(f'---------------{Title}--------------------')
print(f'Price : {Price}')
print(f'24hr % Change : {Change24}')
print(f'Market Cap : {MarketCap}')
print(f'24hr Volume : {Volume}')
print(f'24hr Low : {Low24}')
print(f'24hr High : {High24}')
print('---------------About--------------------')
print(About)
print('---------------Source--------------------')
print(wall_street)

In [ ]:
#LOGO PHOTO
code = str.lower(Title)
snap = f'https://cryptologos.cc/logos/{formed}-{code}-logo.png?v=010'
print(snap)

In [ ]:
#TO BE CONTINUED -- History Scrape
history = f'https://coinmarketcap.com/currencies/{formed}/historical-data/'
#history